In [1]:
import numpy as np
import cv2 as cv
import pandas as pd
import dlib
import os
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
import warnings

# 1. Montando dataframe

In [2]:
def load_dataframe():
    '''
    Carrega um dataframe Pandas com as imagens para o treinamento do modelo
    
    '''
    dados = {
        "File": [],
        "Label": [],
        "Target": [],
        "Image": [],
    }

    open_eyes = os.listdir(f"facialImagens{os.sep}openeye") 
    closed_eyes = os.listdir(f"facialImagens{os.sep}closedeye")

    for arquivo in open_eyes:
        dados["File"].append(f"facialImagens{os.sep}openeye{os.sep}{arquivo}")
        dados["Label"].append(f"Open eye")
        dados["Target"].append(1)
        img = cv.cvtColor(cv.imread(f"facialImagens{os.sep}openeye{os.sep}{arquivo}"), cv.COLOR_BGR2GRAY).flatten()
        dados["Image"].append(img)
        
    for arquivo in closed_eyes:
        dados["File"].append(f"facialImagens{os.sep}closedeye{os.sep}{arquivo}")
        dados["Label"].append(f"Closed eye")
        dados["Target"].append(0)
        img = cv.cvtColor(cv.imread(f"facialImagens{os.sep}closedeye{os.sep}{arquivo}"), cv.COLOR_BGR2GRAY).flatten()
        dados["Image"].append(img)
        
    dataframe = pd.DataFrame(dados)

    return dataframe

In [3]:
dataframe = load_dataframe() #Carregando dataframe com as imagens para treinamento
dataframe

,File,Label,Target,Image
0,facialImagens\openeye\Aaron_Guiel_0001.jpg,Open eye,1,"[7, 10, 14, 17, 19, 24, 30, 36, 42, 43, 46, 49..."
1,facialImagens\openeye\Abdel_Madi_Shabneh_0001.jpg,Open eye,1,"[168, 161, 154, 140, 111, 89, 103, 133, 115, 1..."
2,facialImagens\openeye\Abid_Hamid_Mahmud_Al-Tik...,Open eye,1,"[202, 203, 199, 192, 190, 195, 200, 202, 205, ..."
3,facialImagens\openeye\Adam_Rich_0001.jpg,Open eye,1,"[79, 41, 10, 8, 13, 10, 10, 17, 6, 24, 32, 32,..."
4,facialImagens\openeye\Adolfo_Aguilar_Zinser_00...,Open eye,1,"[93, 118, 125, 107, 99, 110, 114, 105, 107, 10..."
...,...,...,...,...
2418,facialImagens\closedeye\closed_eye_2804.jpg_fa...,Closed eye,0,"[39, 39, 39, 39, 39, 39, 39, 39, 39, 38, 38, 3..."
2419,facialImagens\closedeye\closed_eye_2808.BMP_fa...,Closed eye,0,"[113, 114, 116, 119, 122, 124, 125, 126, 125, ..."
2420,facialImagens\closedeye\closed_eye_2813.BMP_fa...,Closed eye,0,"[38, 14, 21, 57, 79, 85, 89, 90, 57, 60, 68, 8..."
2421,facialImagens\closedeye\closed_eye_2817.jpg_fa...,Closed eye,0,"[74, 66, 55, 45, 41, 40, 37, 32, 18, 22, 24, 2..."


# 2. Divide o dataframe em conjunto de treino e teste

In [4]:
X = list(dataframe["Image"])
y = list(dataframe["Target"])

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.33, random_state=13)

# 3. Carrega o detector de face pelo Dlib

In [5]:
detector = dlib.get_frontal_face_detector() 
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# 4. Função que demarca a região dos olhos por landmarks

In [6]:
def eye_region(eye_points, landmarks):
    eye_region = np.array([(landmarks.part(eye_points[0]).x, landmarks.part(eye_points[0]).y),
                            (landmarks.part(eye_points[1]).x, landmarks.part(eye_points[1]).y),
                            (landmarks.part(eye_points[2]).x, landmarks.part(eye_points[2]).y),
                            (landmarks.part(eye_points[3]).x, landmarks.part(eye_points[3]).y),
                            (landmarks.part(eye_points[4]).x, landmarks.part(eye_points[4]).y),
                            (landmarks.part(eye_points[5]).x, landmarks.part(eye_points[5]).y)], np.int32)
    return eye_region

# 5. Detecta a face e extrai os landmarks

In [7]:
def eye(img, faces):
    for face in faces:
        # Procurando os marcos
        landmarks = predictor(image=img, box=face)
    
        left_eye_region = eye_region([36, 37, 38, 39, 40, 41], landmarks)
        right_eye_region = eye_region([42, 43, 44, 45, 46, 47], landmarks)
        eye = cv.polylines(img, [left_eye_region,right_eye_region], True, 255, 2)
        
    return eye

# 6. extração das features das imagens

In [8]:
#PCA para extração de features das imagen

pca = PCA(n_components=90)
pca.fit(X_train)

PCA(n_components=90)

# 7. Classificação da imagens 

In [9]:
warnings.filterwarnings("ignore")

# Modelo K-Nearest Neighbors

grid_params = {"n_neighbors": [2, 3, 5, 11, 19, 23, 29],
               "weights": ["uniform", "distance"],
               "metric": ["euclidean", "manhattam", "cosine", "l1", "l2"]
              }

knn_model = GridSearchCV(KNeighborsClassifier(), grid_params, refit=True)

knn_model.fit(X_train, y_train)

# predizendo o teste
y_pred = knn_model.predict(X_test)

# comparando predição com o real
print(classification_report(y_test, y_pred)) # precision, recall, f1-score, support

              precision    recall  f1-score   support

           0       0.74      0.56      0.64       799
           1       0.65      0.82      0.73       825

    accuracy                           0.69      1624
   macro avg       0.70      0.69      0.68      1624
weighted avg       0.70      0.69      0.68      1624

